<a href="https://colab.research.google.com/github/sb2539/AI-study/blob/master/GAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install pyts
!pip install einops

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
import matplotlib.pyplot as plt
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import pylab as plt
import numpy as np
import pandas as pd
from pyts.image import RecurrencePlot, GramianAngularField, MarkovTransitionField
from pyts.datasets import fetch_ucr_dataset, ucr_dataset_list
from PIL import Image
from skimage.transform import resize
import os
from torch.utils.tensorboard import SummaryWriter
from einops.layers.torch import Rearrange, Reduce
import copy

In [ ]:
in_features = 3 # 입력 feature (F)
out_features = 3  # 노드의 feature 수 (F프라임)
nb_nodes = 3   # 노드 개수

# node에 대해서 W로 linear transform
W = nn.Parameter(torch.zeros(size = (in_features, out_features )))
nn.init.xavier_uniform_(W, gain=1.414)

input = torch.rand(nb_nodes, in_features) #(3,3)
wh = torch.mm(input, W)
N = wh.size()[0]

print(wh.shape ,"\n", N) 

In [ ]:
a = nn.Parameter(torch.zeros(size = (2*out_features, 1)))
nn.init.xavier_uniform_(a.data, gain=1.414)
print(a.shape)
leakyrelu = nn.LeakyReLU(0.2)  # LeakyReLU

In [ ]:
a_input = torch.cat([wh.repeat(1,N).view(N*N,-1),wh.repeat(N,1)]).view(N, -1, 2*out_features)

In [ ]:
x = torch.tensor([[1,2,3],
                 [4,5,6],
                 [7,8,9]])
print(x,'\n')
y = x.repeat(1,N)
print(y,'\n')
y_t = x.repeat(N,1)
print(y_t,'\n')
z = y.view(N*N,-1)
print(z,'\n')
c = torch.cat([z,y_t],dim=1)
print(c,'\n')
b = c.view(N, -1, 2*out_features)
print(b,'\n')

In [ ]:
e = leakyrelu(torch.matmul(a_input, a).squeeze(2))

In [ ]:
# Masked Attention
adj = torch.randint(2, (3, 3))

zero_vec  = -9e15*torch.ones_like(e)
print(zero_vec.shape)
print(zero_vec)
print(adj.shape)
print(adj)

In [ ]:
attention = torch.where(adj>0, e, zero_vec)
print(adj,"\n",e,"\n",zero_vec)
attention

In [ ]:
attention = F.softmax(attention, dim = 1)
h_prime = torch.matmul(attention, wh)

In [ ]:
name_data = 'Cora'# nodes : 2708, edges : 10,556, features = 1,433, classes : 7
dataset = Planetoid(root= '/tmp/' + name_data, name = name_data)
dataset.transform = T.NormalizeFeatures()

In [ ]:
data = dataset[0]

In [ ]:
data

In [ ]:
data.x[0].size()

In [ ]:
data.train_mask

In [ ]:
wht = torch.tensor([[1,2,3],
                 [4,5,6],
                 [7,8,9]])
at = torch.tensor([[0,1,0],
                 [1,0,1],
                 [1,1,0]])
t_prime = torch.matmul(at, wht)
print(t_prime)

In [ ]:
beef = fetch_ucr_dataset('Beef', use_cache=False, data_home = None, return_X_y=False)
beef_train = beef.data_train
beef_target_train = beef.target_train
beef_test = beef.data_test
beef_target_test = beef.target_test

In [ ]:
df_y = pd.DataFrame(data = beef_target_train)
df_x = pd.DataFrame(data = beef_train)
df_x = df_x.transpose()
df_x.shape

In [ ]:
class GATLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GATLayer, self).__init__()
        self.dropout       = dropout        # drop prob = 0.6
        self.in_features   = in_features    # 
        self.out_features  = out_features   # 
        self.alpha         = alpha          # LeakyReLU with negative input slope, alpha = 0.2
        self.concat        = concat         # conacat = True for all layers except the output layer.

        
        # Xavier Initialization of Weights
        # Alternatively use weights_init to apply weights of choice 
        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        
        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)
        
        # LeakyReLU
        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, input, adj):
        # Linear Transformation
        h = torch.mm(input, self.W) # matrix multiplication
        N = h.size()[0]
        print(N)

        # Attention Mechanism
        a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * self.out_features)
        e       = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))

        # Masked Attention
        zero_vec  = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)
        
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime   = torch.matmul(attention, h)

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime